In [9]:
from __future__ import division

# imports...
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
import numpy as np
import itertools
import math

# Retrieving Hotel Coordinates from Attraction Distances

Exercise: Given the coordinates of 3 (or 4, if needed) attractions, a hotel, and the distances from the hotel to each of the attractions, what are the coordinates of the hotel? First, let's define a couple of functions to calculate just that. We pick 3 (or 4) attractions to begin with, and keep them regardless of the hotel whose coordinates we wish to calculate.

In [44]:
EPS = 0.000001

def get_intersection_point(centers, radii):    
    # store x and y dimensions and radius for easy-to-read code 
    x0, x1, x2 = centers[0][0], centers[1][0], centers[2][0]
    y0, y1, y2 = centers[0][1], centers[1][1], centers[2][1]
    r0, r1, r2 = radii[0], radii[1], radii[2]
    
    # store distances between circle centers (for circle 0 and 1)
    dx = x1 - x0
    dy = y1 - y0
        
    # determine the straight-line distance between the two centers
    dist = math.sqrt(dy ** 2 + dx ** 2)
    
    print dist, r0, r1
    
    # check for solutions for the 2-circle case (do these circles intersect or does one contain the other?)
    if dist > r0 + r1:
        return None
    if dist < abs(r0 - r1):
        return None
    
    # calculate distance from the line through the circle intersection points and the line between the circle centers
    a = (r0 ** 2 - r1 ** 2 + dist ** 2) / (2.0 * dist)
    
    # determine coordinates of this point
    point_x = x0 + dx * (a / dist)
    point_y = y0 + dy * (a / dist)
    
    # determine distance from this point to either of the intersection points
    h = math.sqrt(r0 ** 2 - a ** 2)
    
    # determine the offsets of the intersection points from this point
    rx = -dy * (h / dist)
    ry = dx * (h / dist)
    
    # determine the absolute intersection points
    intersection1_x = point_x + rx
    intersection2_x = point_x - rx
    intersection1_y = point_y + ry
    intersection2_y = point_y - ry
    
    # determine if circle 3 intersects at either of the above intersection points
    dx = intersection1_x - x2
    dy = intersection1_y - y2
    d1 = math.sqrt(dy ** 2 + dx ** 2)
    
    dx = intersection2_x - x2
    dy = intersection2_y - y2
    d2 = math.sqrt(dy ** 2 + dx ** 2)
        
    # check for intersection
    if abs(d1 - r2) < EPS:
        return intersection1_x, intersection1_y
    elif abs(d2 - r2) < EPS:
        return intersection2_x, intersection2_y
    return None  

def get_hotel_coords(attr_coords, distances):
    # try each permutation of the distances
    for perm in itertools.permutations(distances):
        # calculate intersection point
        intersection = get_intersection_point(attr_coords, perm)
        # could come back as NoneType; we check for this here
        if intersection:
            return intersection
        
def get_radius(cx, cy, px, py):
    dx = px - cx
    dy = py - cy
    return math.sqrt(dx ** 2 + dy ** 2)

## Testing the Function

In [46]:
# specifying addresses of three attractions in NYC
attr1 = '1681 Broadway, New York, NY 10019' # Broadway Theatre
attr2 = '1000 5th Ave, New York, NY 10028' # The Metropolitan Museum of Art
attr3 = '350 5th Ave, New York, NY 10118' # Empire State Building

# creating geolocator object
geolocator = Nominatim()

# getting locations of attractions
loc1, loc2, loc3 = geolocator.geocode(attr1), geolocator.geocode(attr2), geolocator.geocode(attr3)

# storing the latitude and longitude of each attraction
coords = [ (loc1.latitude, loc1.longitude), (loc2.latitude, loc2.longitude), (loc3.latitude, loc3.longitude) ]

# specifying address of a hotel in NYC
hotel_address = '768 5th Ave, New York, NY 10019' # The Plaza, NYC Hotel

# getting hotel coords
hotel_loc = geolocator.geocode(hotel_address)
hotel_coords = (hotel_loc.latitude, hotel_loc.longitude)

# get distances from the hotel to each attraction (in miles)
dist1 = get_radius(hotel_coords[0], hotel_coords[1], coords[0][0], coords[0][1])
dist2 = get_radius(hotel_coords[0], hotel_coords[1], coords[1][0], coords[1][1])
dist3 = get_radius(hotel_coords[0], hotel_coords[1], coords[2][0], coords[2][1])

distances = dist1, dist2, dist3

# pass coordinates of the attractions and the calculates hotel -> attraction distances to hotel-finding function
found_hotel_coords = get_hotel_coords(coords, distances)
print 'found vs. actual hotel coordinates:', '\n' + str(found_hotel_coords), '\n' + str(hotel_coords)

# get corresponding address from found coordinates
found_hotel_address = geolocator.reverse(found_hotel_coords)
print 'found vs. actual hotel address:', '\n' + found_hotel_address, '\n' + hotel_address

0.0160455 0.0199909312151
0.025633872144 0.00893091325691 0.0186484864596
0.0195506265181 0.0239783119986 0.0195506265181
found vs. actual hotel coordinates: 
(40.76447735, -73.9744894761588) 
(40.76447735, -73.9744894761588)
found vs. actual hotel address:

TypeError: cannot concatenate 'str' and 'Location' objects